In [1]:
import numpy as np
from scipy import linalg

# 1. Cholesky Decomposition (```linalg.cholesky```)

$$
A = U^\top U, \quad A = U^H U
$$
$$
A = LL^\top , \quad A = LL^H
$$

$A$ is **positive definite (symmetric or Hermitian)** matrix. ($^H$: conjugate transpose)


Cholesky decomposition is numerically stable without pivoting unlike in the case of LU decomposition.\
It is known that the computational speed is about twice that of LU decomposition.

The resulting triangular matrices share the same shape with the corresponding part of original matrix $A$, that is, for example,

$$
\begin{bmatrix}
1 & -1 & 0 \\
-1 & 2 & -1 \\
0 & -1 & 3
\end{bmatrix}
\rightarrow 
U =
\begin{bmatrix}
x & x & 0 \\
0 & x & x \\
0 & 0 & x
\end{bmatrix}
$$

```python
U = linalg.cholesky(A, lower=False) # default
L = linalg.cholesky(A, lower=True) 
```

* ```L``` - Lower Triangular Matrix
* ```U``` - Upper Triangular Matrix
* corresponding Lapack function: ```potrf```

# 2. Cholesky Decomposition Solver (```linalg.cho_solve```)

$$
A\mathbf{x} = \mathbf{b}
$$

$$
A = U^\top U
$$


```python
x = linalg.cho_solve((U, False), b)
x = linalg.cho_solve((L, True), b)
```

* ```U``` or ```L``` - L and U are computed from ```linalg.cholesky```
* corresponding Lapack function: ```potrs```

Example:

$$
A_1 = 
\begin{bmatrix}
1 & -2j  \\
2j & 5  \\
\end{bmatrix}, \;
\mathbf{b}_1 = 
\begin{bmatrix}
1 \\
1  \\
\end{bmatrix}
$$

$$
A_2 = 
\begin{bmatrix}
1 & -1 & 0  \\
-1 & 2 & -1 \\
0 & -1 & 3 \\
\end{bmatrix}, \;
\mathbf{b}_2 = 
\begin{bmatrix}
1 \\
1  \\
1\\
\end{bmatrix}
$$

In [2]:
A1 = np.array([
    [1, -2j],
    [2j, 5]
], dtype=np.complex128)

b1 = np.ones((2, ), dtype = np.float64)

A2 = np.array([
    [1, -1, 0],
    [-1, 2, -1],
    [0, -1, 3]
])

b2 = np.ones((3, ), dtype = np.float64)

In [3]:
""" Example 1. """

# Cholesky decomposition
U1 = linalg.cholesky(A1, lower=False)
print('Example 1.')
print('U_1:')
print(f'{U1}')
print()

# Decomposition result check
print(f'U_1^T U_1 = A_1? --> {np.allclose(U1.T.conjugate() @ U1, A1)}')
print()

# Solve
x1 = linalg.cho_solve((U1, False), b1)
print(f'x1: {x1}')

# solution result check
print(f'Ax = b? --> {np.allclose(A1 @ x1, b1)}')

Example 1.
U_1:
[[ 1.+0.j -0.-2.j]
 [ 0.+0.j  1.+0.j]]

U_1^T U_1 = A_1? --> True

x1: [5.+2.j 1.-2.j]
Ax = b? --> True


In [4]:
""" Example 2. """

# Cholesky decomposition
U2 = linalg.cholesky(A2, lower=False)
print('Example 2.')
print('U_2:')
print(f'{U2}')
print()

# Decomposition result check
print(f'U_2^T U_2 = A_2? --> {np.allclose(U2.T.conjugate() @ U2, A2)}')
print()

# Solve
x2 = linalg.cho_solve((U2, False), b2)
print(f'x2: {x2}')

# solution result check
print(f'Ax = b? --> {np.allclose(A2 @ x2, b2)}')

Example 2.
U_2:
[[ 1.         -1.          0.        ]
 [ 0.          1.         -1.        ]
 [ 0.          0.          1.41421356]]

U_2^T U_2 = A_2? --> True

x2: [4.5 3.5 1.5]
Ax = b? --> True


## 2.1 LU decomposition solver VS Cholesky decomposition

Actually... LU decomposition solver is faster.

When we consider decomposition itself, Cholesky decomposition is faster. But when we are getting the solution for matrix equation after decomposition, LU decomposition solver is faster. 

When it comes to LU decomposition, the diagonal entries of $L$ is all $1$; so it does not compute matrix multiplication regarding the very diagonal entries. That's why LU decomposition solver is faster.

Yet, there are cases when you better use Cholesky decomposition. So, how do we summarize this?

**Conclusion**

1. $A$ is fixed, $\mathbf{b}$ changes a lot $\rightarrow$ Use LU decomposition solver
2. $A$ changes, or a single problem $\rightarrow$ Use Cholesky decomposition solver

Overall, if you do not want to complicate the matters, generally use LU decomposition. But this is not always the case...

# 3. Cholesky Decomposition for Band Matrix (```linalg.cholesky_banded```)

```python
U_band = linalg.cholesky_banded(band_A_h, lower=False) #default #upper band form
L_band = linalg.cholesky_banded(band_A_h, lower=True) #lower band form
```

* ```U_band``` or ```L_band``` - L and U in **banded forms**
* corresponding Lapack function: ```pbtrf```

$$
A =
\begin{bmatrix}
9 & -1 & j & 0 & 0 \\
-1 & 8 & -2 & 2 & 0 \\
-j & -2 & 7 & 3 & 3j \\
0 & 2 & 3 & 6 & 4 \\
0 & 0 & -3j & 4 & 9 \\
\end{bmatrix}, \; \quad
\tilde{A}_{upper} =
\begin{bmatrix}
0 & 0 & j & 2 & 3j \\
0 & -1 & -2 & 3 & 4 \\
9 & 8 & 7 & 6 & 9 \\
\end{bmatrix}, \; \quad
\tilde{A}_{lower} =
\begin{bmatrix}
9 & 8 & 7 & 6 & 9 \\
-1 & -2 & 3 & 4 & 0\\
-j & 2 & -3j & 0 & 0 \\
\end{bmatrix}
$$


$$
\tilde{U}_{upper} =
\begin{bmatrix}
0 & 0 & \colorbox{lightgrey}{$x$} & \colorbox{lightgrey}{$x$} & \colorbox{lightgrey}{$x$} \\
0 & \colorbox{grey}{$x$} & \colorbox{grey}{$x$} & \colorbox{grey}{$x$} & \colorbox{grey}{$x$} \\
\colorbox{pink}{$x$} & \colorbox{pink}{$x$} & \colorbox{pink}{$x$} & \colorbox{pink}{$x$} & \colorbox{pink}{$x$} \\
\end{bmatrix}
$$

* row1: ```Ur1 = U_band[0, 2:]```
* row2: ```Ur2 = U_band[1, 1:]```
* row3: ```Ur3 = U_band[2, 0:]```

Then, $U$ would be:

```U = np.diag(Ur1, k=2) + np.diag(Ur2, k=1) + np.diag(Ur3, k=0)```

$$
U =
\begin{bmatrix}
\colorbox{pink}{$x$} & \colorbox{grey}{$x$} & \colorbox{lightgrey}{$x$} & 0 & 0 \\
0 & \colorbox{pink}{$x$} & \colorbox{grey}{$x$} & \colorbox{lightgrey}{$x$} & 0 \\
0 & 0 & \colorbox{pink}{$x$} & \colorbox{grey}{$x$} & \colorbox{lightgrey}{$x$} \\
0 & 0 & 0 & \colorbox{pink}{$x$} & \colorbox{grey}{$x$} \\
0 & 0 & 0 & 0 & \colorbox{pink}{$x$} \\
\end{bmatrix}
$$

Moreover, the actual $A$ would be:
```A = U.T.conjugate() @ U```

Of course, keep in mind that reconstructing actual $U$ and $A$ is **not recommended** and above was presented *only for educational purposes*.

# 4. Cholesky Decomposition Solver for Band Matrix (```linalg.cho_solve_banded```)

$$
A\mathbf{x} = \mathbf{b}
$$

```python
x = linalg.cho_solve_banded((U_band, False), b) #default #upper band form
x = linalg.cho_solve_banded((L_band, True), b) #lower band form
```

* ```U_band``` or ```L_band``` - L and U in **banded forms** from ```linalg.cholesky_banded```
* corresponding Lapack function: ```pbtrs```

$$
A =
\begin{bmatrix}
9 & -1 & j & 0 & 0 \\
-1 & 8 & -2 & 2 & 0 \\
-j & -2 & 7 & 3 & 3j \\
0 & 2 & 3 & 6 & 4 \\
0 & 0 & -3j & 4 & 9 \\
\end{bmatrix}, \; \quad
\tilde{A}_{upper} =
\begin{bmatrix}
0 & 0 & j & 2 & 3j \\
0 & -1 & -2 & 3 & 4 \\
9 & 8 & 7 & 6 & 9 \\
\end{bmatrix}, \; \quad
\mathbf{b} =
\begin{bmatrix}
1 \\
1 \\
1 \\
1 \\
1 \\
\end{bmatrix}
$$

In [5]:
A_band_h = np.array([
    [0, 0, 1j, 2, 3j],
    [0, -1, -2, 3, 4],
    [9, 8, 7, 6, 9]
], dtype=np.complex128)

b = np.ones((5, ), dtype=np.float64)

In [6]:
U_band = linalg.cholesky_banded(A_band_h, lower=False)
print(U_band)

[[ 0.        +0.j          0.        +0.j          0.        +0.33333333j
   0.71206899+0.j          0.        +1.18768515j]
 [ 0.        +0.j         -0.33333333+0.j         -0.71206899+0.03955939j
   1.38842067+0.01115197j  2.11145768-0.87334379j]
 [ 3.        +0.j          2.80871659+0.j          2.52592195+0.j
   1.88815291+0.j          1.53896753+0.j        ]]


In [7]:
""" educational purpose only """
Ur1 = U_band[0, 2:]
Ur2 = U_band[1, 1:]
Ur3 = U_band[2, :]
U = np.diag(Ur1, k=2) + np.diag(Ur2, k=1) + np.diag(Ur3, k=0)
# print(U)

A = U.T.conjugate() @ U
# print(A)
""" """

' '

In [8]:
x = linalg.cho_solve_banded((U_band, False), b)
print(x)

[ 0.1524183 -0.06901961j  0.44026144+0.01385621j  0.63503268-0.06849673j
 -0.54980392-0.15843137j  0.37830065+0.2820915j ]


In [9]:
print(np.real_if_close(A @ x)) # b

[1. 1. 1. 1. 1.]


# 5. Strategy for Matrix Decompositions

```mermaid
graph TD
    A[Real symmetric / complex Hermitian ?]
    A -->|Y| C[Positive definite?]
    A -->|N| D[Complex symmetric?]
    C -->|Y| E[Cholesky Decomposition]
    C -->|N| F[Diagonal Pivoting: 
    LDL^T, LDL^H]
    D -->|Y| G[Diagonal Pivoting: 
    LDL^T]
    D -->|N| H[LU Decomposition]
```

Currently.. with ```Scipy```

1. Cholesky Decomposition - solver o, band matrix specialized function o
2. LU Decomposition - solver o, band matrix specialized function x
3. Diagonal pivoting - solver x, band matrix specialized function x $\rightarrow$ just use LU decomposition or use general solver with ```assume_a="sym"``` or ```"her"```

$\Rightarrow$ Use LAPACK directly for LU decomposition of band matrices.

### Practice

In [10]:
import timeit

Example:

$$
A =
\begin{bmatrix}
5 & 1 &   j     &        &      &  \\
1 & 5 & 1     &  j      &       & \\
-j   & 1 & 5     & \ddots & \ddots &       \\
   &  \ddots  & \ddots & \ddots & 1  & j   \\
      &    &     -j   & 1     & 5& 1\\
   &    &        & -j     & 1 &5
\end{bmatrix}_{10000 \times 10000}
$$

In [11]:
off_diag1 = np.ones((9999, ))
off_diag2 = np.ones((9998, ))
A = 5*np.identity(10000) + np.diag(off_diag1, k=1) + np.diag(off_diag1, k=-1) + np.diag(off_diag2, k=2) * 1j + np.diag(off_diag2, k=-2) * 1j

In [12]:
row0 = np.hstack((np.ones((2, )), off_diag2))
row1 = np.hstack((np.ones((1, )), off_diag1))
row2 = np.diag(A)
A_band_h = np.vstack((row0, row1, row2))

In [13]:
b = np.ones((10000, ))

**1. LU Decomposition with full matrix**

In [14]:
start = timeit.default_timer()

lu, piv = linalg.lu_factor(A)

end = timeit.default_timer()

print('LU decomposition: ')
print(f'{end-start: 6f}')

LU decomposition: 
 14.090051


In [15]:
start = timeit.default_timer()

x = linalg.lu_solve((lu, piv), b)

end = timeit.default_timer()

print('LU Solver: ')
print(f'{end-start: 6f}')

LU Solver: 
 0.231423


**2. Cholesky Decomposition with full matrix**

In [16]:
start = timeit.default_timer()

U = linalg.cholesky(A, lower=False)

end = timeit.default_timer()

print('Cholesky decomposition: ')
print(f'{end-start: 6f}')

Cholesky decomposition: 
 10.040177


In [17]:
start = timeit.default_timer()

x = linalg.cho_solve((U, False), b)

end = timeit.default_timer()

print('Cholesky solver: ')
print(f'{end-start: 6f}')

Cholesky solver: 
 0.994747


* Decomposition - Cholesky is faster
* Solve - LU is faster (taking advantage of the fact that the diagnal entries of $L$ is all $1$)

**3. Cholesky Decomposition with band matrix**

In [18]:
start = timeit.default_timer()

U_band = linalg.cholesky_banded(A_band_h, lower=False)

end = timeit.default_timer()

print('Cholesky decomposition (band): ')
print(f'{end-start: 6f}')

Cholesky decomposition (band): 
 0.024177


In [19]:
start = timeit.default_timer()

x = linalg.cho_solve_banded((U_band, False), b)

end = timeit.default_timer()

print('Cholesky solver (band): ')
print(f'{end-start: 6f}')

Cholesky solver (band): 
 0.047079


Using band matrix is vastly faster -> sparse matrix

Next content - how to take advantage of band matrix in LU decomposition? -> directly use low-level LAPACK functions